<a href="https://colab.research.google.com/github/anujgupta82/NLP_Bootcamp/blob/V_2_0/module2/BiGram/BiGram_barebones_1_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------

# Gentle introduction to BiGram model

----------

### Objective is correctness and clarity of basic concepts and not efficiency

#### Author : Anuj

#### Uses Wikipedia Dataset

In [0]:
import sys
import tensorflow as tf
import numpy as np
import random
import itertools

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
sys.path.append("/content/drive/GoogleDrive_Utils/")

In [0]:
from readWikiData import get_wikipedia_data

In [0]:
def get_wiki_data(n_vocab_=1000):
    sentences, word2idx, _, _ = get_wikipedia_data(n_vocab=n_vocab_, n_files=10, by_paragraph=True)
    training_data = []
    vocab_size = len(word2idx)
    for sentence in sentences:
        for elem1, elem2 in zip(sentence[:-1], sentence[1:]):
            training_data.append((elem1, elem2))
    
    # this destroys the order of words in a wondow but for bigram its harmless
    # all we want is - pair of all bigrams
    training_data = list(set(training_data))   
    
    idx2word = {v:k for k, v in word2idx.items()}
    return len(word2idx), training_data, word2idx, idx2word

In [0]:
vocab_size, training_data, word2idx, idx2word = get_wiki_data(n_vocab_=9999)

In [0]:
print (vocab_size)
print (type(training_data))

10000
<class 'list'>


In [0]:
training_data[:10]

[(1760, 65),
 (197, 1097),
 (381, 392),
 (3174, 1259),
 (9999, 5827),
 (7926, 433),
 (312, 241),
 (803, 2857),
 (784, 1839),
 (25, 701)]

##### Get batches

In [0]:
# contains list of pairs that have already been selected
bucket_list = []

def getNextBatch(bi_grams_, batch_size=1000):
    
    global bucket_list
    
    # list of possible pairs to pick from
    docs_ids_to_select = list(set(bi_grams_) - set(bucket_list))
    
    # once you exhaust the possible pais, reset
    if len(docs_ids_to_select) < batch_size:
        bucket_list = []
        docs_ids_to_select = bi_grams_
        
    # Initialize two variables 
    train_X = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    train_label = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # pick a random chunks of pairs 
    random_docs = random.sample(docs_ids_to_select, batch_size)
    bucket_list += random_docs
    
    index = 0 
    
    # Iterate threw all the docs 
    for item in random_docs:
        train_X[index] = item[0]
        train_label[index] = item[1]  
        index += 1
        
    #flatten list of lists to a single list
    train_X = list(itertools.chain(*train_X))
    train_label = list(itertools.chain(*train_label))
            
    return train_X, train_label

In [0]:
X, Y = getNextBatch(bi_grams_=training_data, batch_size=32)

In [0]:
print (len(X), len(Y))

32 32


In [0]:
print (X[:10], Y[:10])

[78, 2657, 6185, 2904, 587, 33, 6106, 3677, 6850, 2780] [186, 1755, 2432, 527, 3309, 1059, 56, 202, 4, 10]


# Network

In [0]:
batch_size=32
num_batches = len(training_data)/batch_size

print ("Number of batches = %d" %num_batches)

embedding_dims = 128



Number of batches = 52019


In [0]:
X = tf.placeholder(shape=(batch_size,), dtype = tf.int32)
Y = tf.placeholder(shape=(batch_size,), dtype = tf.int32)

In [0]:
y_oh = tf.one_hot(indices=X, depth=vocab_size)

In [0]:
print (X.get_shape())
print (Y.get_shape())
print (y_oh.get_shape())

(32,)
(32,)
(32, 10000)


In [0]:
embedding_layer_1 = tf.Variable(tf.truncated_normal(shape=(vocab_size, embedding_dims), 
                                                    mean=0.0, stddev=1.0, dtype=tf.float32))
embeded = tf.nn.embedding_lookup(embedding_layer_1, ids=X)

In [0]:
embeded.get_shape()

TensorShape([Dimension(32), Dimension(128)])

In [0]:
#softmax weights, bias
W = tf.Variable(tf.truncated_normal(shape=(embedding_dims, vocab_size),mean=0.0, stddev=1.0, dtype=tf.float32))
b = tf.Variable(tf.zeros(shape=(vocab_size,)))


In [0]:
logits = tf.add(tf.matmul(embeded, W), b )

#define loss
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_oh)
mean_loss = tf.reduce_mean(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [0]:
print (logits.get_shape())
print (y_oh.get_shape())

(32, 10000)
(32, 10000)


In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(mean_loss)

In [0]:
num_of_epochs = 20

#batch = BatchData(batch_size=32, list_of_token_ids=data)

with tf.Session() as sess:

    tf.global_variables_initializer().run()

    for epoch_id in range(num_of_epochs):

        av_batch_loss = 0

        for batch_id in range(int(num_batches)):

            X_, Y_ = getNextBatch(bi_grams_=training_data, batch_size=batch_size)

            feed_dict = {}
            feed_dict[X] = X_
            feed_dict[Y] = Y_

            batch_loss, _ = sess.run([mean_loss, optimizer], feed_dict=feed_dict)

            av_batch_loss += batch_loss
            
            if batch_id % 100 == 0:
                print ("For epoch = %d, batch id = %d, batch loss = %f" %(epoch_id, batch_id, batch_loss))

        print ("For epoch = %d, Av loss = %f" %(epoch_id, av_batch_loss/num_batches))
        
        batch.reset()
        
        
        
        
        
        

For epoch = 0, batch id = 0, batch loss = 34.728737
